Para esta iteracion, probaremos con distintas formas de segmentacion sobre nuestro dataset para mejorar la capacidad de prediccion de nuestro modelo.

Buildeamos codigo C++
Mover el build generado en la carpeta experimentacion a la carpeta requerida(en este caso experimentacion_precio)

In [ ]:
!cd ../../ && mkdir build
!cd ../../build/ && rm -rf *
!cd ../../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../../build && make install

Importamos dependencias

In [1]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import ModeloM2
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_error
from pathlib import Path

Importamos el dataset a utilizar

In [17]:
df = pd.read_csv('../../data/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          240000 non-null  int64  
 1   titulo                      234613 non-null  object 
 2   descripcion                 238381 non-null  object 
 3   tipodepropiedad             239954 non-null  object 
 4   direccion                   186928 non-null  object 
 5   ciudad                      239628 non-null  object 
 6   provincia                   239845 non-null  object 
 7   antiguedad                  196445 non-null  float64
 8   habitaciones                217529 non-null  float64
 9   garages                     202235 non-null  float64
 10  banos                       213779 non-null  float64
 11  metroscubiertos             222600 non-null  float64
 12  metrostotales               188533 non-null  float64
 13  idzona        

Definimos un diccionario con dos modelos distintos. Ambos funcionan de la misma manera pero estan codeados de maneras muy distintas. Los resultados mostrados en el informe son utilizando el v2.

In [3]:
modelos = {
    'v2': ModeloM2.ModeloV2,
}

Definimos las columnas seleccionadas, las cuales son la mejor combinacion a la hora de predecir SIN segmentacion.
Definimos las distintas segmentaciones a utilizar en esta prueba.

In [34]:
columnas_seleccionadas = ["centroscomercialescercanos","usosmultiples","banos","habitaciones","precio","metrostotales"]

segmentaciones = [['tipodepropiedad','piscina'], ['tipodepropiedad','piscina', 'escuelascercanas','garages'], ['tipodepropiedad'], ['tipodepropiedad','piscina'], ['tipodepropiedad','escuelascercanas'], ['tipodepropiedad','provincia']]

In [35]:
import warnings
warnings.filterwarnings('ignore')

KFOLD_K = 5
kf = KFold(n_splits=KFOLD_K, shuffle=True)


result = {}
for segmentacion in segmentaciones:
    rms = []
    rmsle = []
    mae = []
    r2 = []
    df = df[df['metroscubiertos'].notna()]
    df.reset_index(drop=True, inplace=True)
    for train_index, test_index in kf.split(df):
        df_train, df_test = df.loc[train_index], df.loc[test_index]
        
        mp = modelos['v2'](df_train, columnas_seleccionadas)
        df_predicted = mp.run(df_test, segmentacion)
    
        
        df_predicted['metroscubiertos'] = df_predicted['metroscubiertos'].copy().apply(lambda x: x if x > 0 else 0) 
        rms.append(mean_squared_error(df_test["metroscubiertos"], df_predicted["metroscubiertos"], squared=False))
        rmsle.append(mean_squared_log_error(df_test["metroscubiertos"],  df_predicted['metroscubiertos']))
        r2.append(r2_score(df_test["metroscubiertos"], df_predicted["metroscubiertos"]))
        mae.append(mean_absolute_error(df_test["metroscubiertos"], df_predicted["metroscubiertos"]))

    rms_result = np.sum(rms) / KFOLD_K 
    rmsle_result = np.sum(rmsle) / KFOLD_K 
    r2_result = np.sum(r2) / KFOLD_K 
    mae_result = np.sum(mae) / KFOLD_K

    result[', '.join(segmentacion)] = [rms_result, rmsle_result, r2_result, mae_result]


out = pd.DataFrame.from_dict(result, orient='index', columns=['RMSE', 'RMSLE', 'R^2', 'MAE']).to_latex(column_format='p{6cm}|r|r|r|r')
print(out)

\begin{tabular}{p{6cm}|r|r|r|r}
\toprule
{} &       RMSE &     RMSLE &       R\textasciicircum 2 &        MAE \\
\midrule
tipodepropiedad, piscina                           &  51.729571 &  0.088868 &  0.722161 &  36.921592 \\
tipodepropiedad, piscina, escuelascercanas, gar... &  54.737710 &  0.088158 &  0.684208 &  35.708493 \\
tipodepropiedad                                    &  51.869070 &  0.089586 &  0.720703 &  37.158942 \\
tipodepropiedad, escuelascercanas                  &  51.690757 &  0.089208 &  0.722628 &  37.052740 \\
tipodepropiedad, provincia                         &  54.120192 &  0.110898 &  0.694940 &  34.622454 \\
\bottomrule
\end{tabular}

